# Guia 6

## Ejercicio 2

Diseñe e implemente un algoritmo genético que busque el mejor subconjunto de características para la clasificación de cáncer (leucemia linfocítica aguda y leucemia mielógena aguda) a partir de datos genómicos. Se proveen 38 muestras en el conjunto de entrenamiento y 34 en el conjunto de prueba (`leukemia_train.csv` y `leukemia_test.csv`, respectivamente). Cada muestra se compone de un total de 7129 características, que corresponden a valores de expresión génica.

### Solución

La solución buscada será una selección de características de entre las 7129 necesarias para que un perceptrón multicapa resuelva el problema, dando mayor aptitud a aquellas con menor número de características (promoviendo la simplicidad de la solución).
Para solución se **codificará** como una cadena de 7129 bits, donde un `1` indicará que la característica es necesaria para solución y un `0` que no.

> Esto que sigue es inchequeable, no lo probe

Para evaluar su aptitud se entrenará un perceptrón multicapa con esas características y los datos de entrenamiento y se medirá su F1. Finalmente, su aptitud estará dada por
$$
f =
\begin{cases}
    -1000 \quad&\text{si }N = 0\\
    \frac{F1}{N} \quad&\text{en otro caso}
\end{cases}
$$
donde $N$ serán la cantidad de características de la solución.

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import os
import numpy as np
import matplotlib.pyplot as plt
from Evol import Evol

def cargarCSV(csvRelativePath, delimeter = ',') -> np.ndarray[any]:
    this_folder = os.path.abspath('') 
    fullPath = os.path.join(this_folder, csvRelativePath)
    return np.loadtxt(fullPath, dtype=float, delimiter=delimeter)

# ===== Leer archivos =====
# leer archivos
trnFileConcent = cargarCSV('./data/leukemia_train.csv')
tstFileConcent = cargarCSV('./data/leukemia_test.csv')

x_trn = trnFileConcent[:,:-1]
yd_trn = trnFileConcent[:,-1]
x_tst = tstFileConcent[:,:-1]
yd_tst = tstFileConcent[:,-1]

In [3]:
# ===== Algoritmo evolutivo =====
# funcion de decodificacion
def b2caracts(x: np.ndarray[np.uint8]) -> list[np.ndarray[int]]|np.ndarray[int]:
    x = np.atleast_2d(x)
    # una poblacion
    N, B = x.shape  # N individuos, B bits por individuo
    caracts = [np.flatnonzero(x[i]) for i in range(N)]
    return caracts if len(caracts) > 1 else caracts[0]

# funcion de binarizacion
def f2b(x: list[float], B: int) -> np.ndarray[np.uint8]:
    x = np.atleast_1d(x)
    norm = x/1024 + 0.5
    ints = np.clip(np.round(norm * (2**B - 1)), 2**B - 1).astype(np.uint64)
    bits = ((ints[:, None] >> np.arange(B-1, -1, -1)) & 1).astype(np.uint8)
    return bits if len(x) > 1 else bits[0]

@ignore_warnings(category=ConvergenceWarning)
def fitness(x: np.ndarray[np.uint8]) -> np.ndarray[float]|float:
    # obtenemos los indices de las caracteristicas
    individuos = b2caracts(x)
    n_individuos = len(individuos)
    fits = np.zeros(n_individuos, dtype=float)
    for i, caracts in enumerate(individuos):
        n_caracts = caracts.shape[0]
        # si hay 0 caracteristicas, penalizar
        if n_caracts == 0:
            fits[i] = -1000
            continue

        # entrenamos y evaluamos el perceptron con las caracteristicas elegidas
        inputs = x_trn[:,caracts]
        mlp = MLPClassifier(
            hidden_layer_sizes=(n_caracts),
            max_iter=5
        )
        mlp.fit(inputs, yd_trn)
        f1 = f1_score(yd_trn, mlp.predict(inputs))
        fits[i] = f1/float(n_caracts)

    return fits if fits.shape[0] > 1 else fits[0]

def init_poblacion(N: int, B: int, rng: np.random.Generator) -> np.ndarray[np.uint8]:
    # tomar 20 posiciones al azar y hacerlas 1 (para cada individuo)
    pob = np.zeros((N,B),dtype=np.uint8)
    posiciones = rng.integers(0,B,size=(N,20))
    for n in range(N):
        pob[n, posiciones[n,:]] = 1
    return pob

evol = Evol(
    init_fun=init_poblacion,
    fit_fun=fitness,
    max_gen=200,
    poblacion=200,
    progenitores=20,
    long_cromosoma=7129,
    mutacion=0.1,
    tolerancia=200
)

poblacion = evol.evolve()

100%|██████████| 200/200 [02:44<00:00,  1.22it/s]


In [4]:
mejor = poblacion[np.argmin(fitness(poblacion))]
mejores_caracteristicas = b2caracts(mejor)
mlp = MLPClassifier(
    hidden_layer_sizes=(mejores_caracteristicas.shape[0]),
    max_iter=50
)
mlp.fit(x_trn[:,mejores_caracteristicas], yd_trn)
f1_trn = f1_score(yd_trn, mlp.predict(x_trn[:,mejores_caracteristicas]))
f1_tst = f1_score(yd_tst, mlp.predict(x_tst[:,mejores_caracteristicas]))

In [5]:
print(mejores_caracteristicas.shape[0])
print(mejores_caracteristicas)
print(f1_trn)
print(f1_tst)

22
[ 352  533  838 1747 1887 2057 2185 2603 3431 3452 3936 4366 5010 5076
 5394 5542 5933 6033 6341 6700 6844 6959]
0.631578947368421
0.5217391304347826
